In this notebook, we firstly extract information about each Idaweb's weather station from a legend. <br>
After that, we create an interactive map showing their position in Switzerland.

# Get all station names and locations

In [1]:
import pandas as pd
import station_info as si

%load_ext autoreload
%autoreload 2

`stations_legend.txt` contains a list of stations and information associated to them (such as their coordinates) for which we have data for all the features needed for training: temperature, wind and irrandiance.

In [2]:
stationsFile = 'stations_legend.txt'

For each station, we retrieve:
- its name
- its coordinates

In [3]:
station_dict = {}
with open(stationsFile) as fp:
    for line in fp:
        name, lat, lng = si.getParsedStation(line)
        station_dict[name] = (lat, lng)

And we create a pd.DataFrame

In [4]:
station_df = pd.DataFrame.from_dict(station_dict, orient='index')
station_df.reset_index(inplace=True)
station_df = station_df.rename(columns = {'index':'name', 0:'lat', 1:'lng'})
station_df.head()

,name,lat,lng
0,KLO,47.483333,8.533333
1,FRU,46.600000,7.650000
2,CEV,46.316667,8.600000
3,EIN,47.133333,8.750000
4,WSLNAF,46.666667,10.233333


In [ ]:
pickle.dump(station_df, open("station_df.p", "wb")) # save DataFrame

# Visualizing the stations

Finally, we represent each station in a map by a marker.

In [5]:
import folium

In [6]:
topo_path = r'ch-cantons.topojson.json' # TopoJSON file with the geo-coordinates of each Swiss canton

In [7]:
swiss_map = folium.Map(location=(46.8182,8.2275), zoom_start=8, tiles='cartodbpositron')

swiss_map.choropleth(
    geo_path=topo_path,topojson='objects.cantons',
    fill_color='red',
    fill_opacity=0.3,
    line_weight=2,
)

swiss_map

In [8]:
# get values from each column in the stations' DataFrame
lats = station_df.lat.get_values()
lngs = station_df.lng.get_values()
names = station_df.name.get_values()

In [9]:
namelatlng = list(zip(names, lats, lngs))

In [10]:
for i,ll in enumerate(namelatlng):
    folium.Marker([ll[1], ll[2]], popup = ll[0] + ' ('+ str(ll[1]) + ',' + str(ll[2]) + ')').add_to(swiss_map)

swiss_map

In [11]:
swiss_map.save('stations_map.html')